<a href="https://colab.research.google.com/github/faisal-fida/100-Python-Projects-in-Google-Colab/blob/main/Alexnet_%26_Cifar_10_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
alexnet = models.alexnet(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 187MB/s]


In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
data_dir = '/content/drive/MyDrive/Google Colab/CV/Assignments/Assignment 5/Dataset/cifar-10-python'
dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=data_transforms)
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

# Set the model to evaluation mode
alexnet.eval()

features = []
labels = []
max_iterations = 10  # Set the maximum number of iterations

with torch.no_grad():
    for i, (inputs, targets) in enumerate(data_loader):
        # Forward pass through the model
        outputs = alexnet(inputs)
        # Extract the features from the second last layer
        feats = alexnet.features[0:12](inputs).view(inputs.size(0), -1)
        # Add the features and labels to the list
        features.append(feats)
        labels.append(targets)

        # Check if maximum iterations reached
        if i+1 >= max_iterations:
            break



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Concatenate the features and labels
features = torch.cat(features, dim=0)
labels = torch.cat(labels, dim=0)

# Save the extracted features and labels
torch.save(features, 'features.pt')
torch.save(labels, 'labels.pt')

train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)


In [ ]:
clf = svm.SVC(kernel='linear')

In [ ]:
# Train SVM classifier
clf.fit(train_data, train_labels)

SVC(kernel='linear')

In [ ]:
pred_labels = clf.predict(test_data)
accuracy = accuracy_score(test_labels, pred_labels)
print("Accuracy:", accuracy)

Accuracy: 0.5729166666666666
